# Sequential Simulations with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways. This
example demonstrates some of these capabilities to represent electricity market clearing.

## Dependencies
Since the `OperatiotnsProblem` is the fundamental building block of a sequential
simulation in PowerSimulations, we will build on the [OperationsProblem example](../../notebook/PowerSimulations_examples/1_operations_problems.ipynb)
by sourcing it as a dependency.

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "PowerSimulations_examples", "1_operations_problems.jl"))

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/s


Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      ILoads:
        device_type = PowerSystems.InterruptibleLoad
        formulation = PowerSimulations.InterruptiblePowerLoad
      HydroROR:
        device_type = PowerSystems.HydroDispatch
        formulation = PowerSimulations.FixedOutput
      Generators:
        device_type = PowerSystems.ThermalStandard
        formulation = PowerSimulations.ThermalStandardUnitCommitment
      Ren:
        device_type = PowerSystems.RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
      Hydro:
        device_type = PowerSystems.HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchRunOfRiver
      Loads:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RenFx:
        device_type = PowerSystems.RenewableFix
        formulation = PowerSimulations.FixedOutput
  branches: 
      T:
        de

### 5-Minute system
We had already created a `sys::System` from hourly RTS data in the OperationsProblem example.
The RTS data also includes 5-minute resolution time series data. So, we can create another
`System`:

In [2]:
sys_RT = System(rawsys; forecast_resolution = Dates.Minute(5))

┌ Warning: Missing LOAD data.
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:212
┌ Warning: Missing LOAD data.
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:212
┌ Info: Adding contributing generators for Spin_Up_R1 by category
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:656
┌ Info: Adding contributing generators for Spin_Up_R2 by category
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:656
┌ Info: Adding contributing generators for Spin_Up_R3 by category
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:656
┌ Info: Adding contributing generators for Flex_Up by category
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:656
┌ Info: Adding contributing gen

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,73
3,GenericBattery,Storage <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
4,HVDCLine,DCBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
5,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
6,HydroEnergyReservoir,HydroGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,19
7,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,105
8,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,21
9,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,51


## `OperationsProblemTemplate`s define `Stage`s
Sequential simulations in PowerSimulations are created by defining `OperationsProblems`
that represent `Stages`, and how information flows between executions of a `Stage` and
between different `Stage`s.

Let's start by defining a two stage simulation that might look like a typical day-Ahead
and real-time electricity market clearing process.

### Define the reference model for the day-ahead unit commitment

In [3]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalStandardUnitCommitment),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, FixedOutput),
    :Hydro => DeviceModel(HydroEnergyReservoir, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, FixedOutput),
)
template_uc = template_unit_commitment(devices = devices)


Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      HydroROR:
        device_type = PowerSystems.HydroDispatch
        formulation = PowerSimulations.FixedOutput
      Generators:
        device_type = PowerSystems.ThermalStandard
        formulation = PowerSimulations.ThermalStandardUnitCommitment
      Ren:
        device_type = PowerSystems.RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
      Hydro:
        device_type = PowerSystems.HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchRunOfRiver
      Loads:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RenFx:
        device_type = PowerSystems.RenewableFix
        formulation = PowerSimulations.FixedOutput
  branches: 
      T:
        device_type = PowerSystems.Transformer2W
        formulation = PowerSimulations.StaticTransformer
      TT:
        device_type = P

### Define the reference model for the real-time economic dispatch

In [4]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatch),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, FixedOutput),
    :Hydro => DeviceModel(HydroEnergyReservoir, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, FixedOutput),
)
template_ed = template_economic_dispatch(devices = devices)


Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      HydroROR:
        device_type = PowerSystems.HydroDispatch
        formulation = PowerSimulations.FixedOutput
      Generators:
        device_type = PowerSystems.ThermalStandard
        formulation = PowerSimulations.ThermalDispatch
      Ren:
        device_type = PowerSystems.RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
      Hydro:
        device_type = PowerSystems.HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchRunOfRiver
      Loads:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RenFx:
        device_type = PowerSystems.RenewableFix
        formulation = PowerSimulations.FixedOutput
  branches: 
      T:
        device_type = PowerSystems.Transformer2W
        formulation = PowerSimulations.StaticTransformer
      TT:
        device_type = PowerSystems.Ta

### Define the `Stage`s
Stages define models. The actual problem will change as the stage gets updated to represent
different time periods, but the formulations applied to the components is constant within
a stage. In this case, we want to define two stages with the `OperationsProblemTemplate`s
and the `System`s that we've already created.

In [5]:
stages_definition = Dict(
    "UC" => Stage(GenericOpProblem, template_uc, sys, solver),
    "ED" => Stage(
        GenericOpProblem,
        template_ed,
        sys_RT,
        solver,
        balance_slack_variables = true,
    ),
)

Dict{String,PowerSimulations.Stage{PowerSimulations.GenericOpProblem}} with 2 entries:
  "ED" => Stage()…
  "UC" => Stage()…

Note that the "ED" stage has a `balance_slack_variables = true` argument. This adds slack
variables with a default penalty of 1e6 to the nodal energy balance constraint and helps
ensure feasibility with some performance impacts.

### `SimulationSequence`
Similar to an `OperationsProblemTemplate`, the `SimulationSequence` provides a template of
how to execute a sequential set of operations problems.

print_struct(SimulationSequence)

Let's review some of the `SimulationSequence` arguments.

### Chronologies
In PowerSimulations, chronologies define where information is flowing. There are two types
of chronologies.
 - inter-stage chronologies: Define how information flows between stages. e.g. day-ahead
solutions are used to inform economic dispatch problems
 - intra-stage chronologies: Define how information flows between multiple executions of a
single stage. e.g. the dispatch setpoints of the first period of an economic dispatch problem
are constrained by the ramping limits from setpoints in the final period of the previous problem.

Let's define an inter-stage chronology that synchronizes information from 24 periods of
the first stage with a set of executions of the second stage:

In [6]:
feedforward_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24))

Dict{Pair{String,String},PowerSimulations.Synchronize} with 1 entry:
  "UC"=>"ED" => Synchronize(24)

### `FeedForward` and `Cache`
The definition of exactly what information is passed using the defined chronologies is
accomplished with `FeedForward` and `Cache` objects. Specifically, `FeedForward` is used
to define what to do with information being passed with an inter-stage chronology. Let's
define a `FeedForward` that affects the semi-continuous range constraints of thermal generators
in the economic dispatch problems based on the value of the unit-commitment variables.

In [7]:
feedforward = Dict(
    ("ED", :devices, :Generators) => SemiContinuousFF(
        binary_source_stage = PSI.ON,
        affected_variables = [PSI.ACTIVE_POWER],
    ),
)

Dict{Tuple{String,Symbol,Symbol},PowerSimulations.SemiContinuousFF} with 1 entry:
  ("ED", :devices, :Generators) => SemiContinuousFF(:On, [:P], nothing)

### Sequencing
The stage problem length, look-ahead, and other details surrounding the temporal Sequencing
of stages are controlled using the `order`, `horizons`, and `intervals` arguments.
 - order::Dict(Int, String) : the hierarchical order of stages in the simulation
 - horizons::Dict(String, Int) : defines the number of time periods in each stage (problem length)
 - intervals::Dict(String, Dates.Period) : defines the interval with which stage problems
advance after each execution. e.g. day-ahead problems have an interval of 24-hours

So, to define a typical day-ahead - real-time sequence, we can define the following:
 - Day ahead problems should represent 48 hours, advancing 24 hours after each execution (24-hour look-ahead)
 - Real time problems should represent 1 hour (12 5-minute periods), advancing 1 hour after each execution (no look-ahead)

In [8]:
order = Dict(1 => "UC", 2 => "ED")
horizons = Dict("UC" => 24, "ED" => 12)
intervals = Dict("UC" => (Hour(24), Consecutive()), "ED" => (Hour(1), Consecutive()))

Dict{String,Tuple{Dates.Hour,PowerSimulations.Consecutive}} with 2 entries:
  "ED" => (1 hour, Consecutive())
  "UC" => (24 hours, Consecutive())

Finally, we can put it all together:

In [9]:
DA_RT_sequence = SimulationSequence(
    step_resolution = Hour(24),
    order = order,
    horizons = horizons,
    intervals = intervals,
    ini_cond_chronology = InterStageChronology(),
    feedforward_chronologies = feedforward_chronologies,
    feedforward = feedforward,
)

Feed Forward Chronology
-----------------------

ED: PowerSimulations.SemiContinuousFF -> Generators

                     UC--┐ from : On
                         |
┌----┬----┬----┬----┬----┼----┬----┬----┬----┬----┬----┐
|    |    |    |    |    |    |    |    |    |    |    |
|    |    |    |    |    |    |    |    |    |    |    |
└─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED ... (x24) to : ["P"]

Initial Condition Chronology
----------------------------

1
|
|
2 --> 2 ... (x24)   


## `Simulation`
Now, we can build and execute a simulation using the `SimulationSequence` and `Stage`s
that we've defined.

In [10]:
sim = Simulation(
    name = "rts-test",
    steps = 1,
    stages = stages_definition,
    stages_sequence = DA_RT_sequence,
    simulation_folder = rts_dir,
)

Simulation()


### Build simulation

In [11]:
build!(sim)

### Execute simulation

In [12]:
sim_results = execute!(sim)

Executing Step 1
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: May 23 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Continuous objective value is 1.06949e+06 - 0.43 seconds
Cgl0003I 58 fixed, 0 tightened bounds, 1450 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 105 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1392 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1288 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1159 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 558 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 526 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 468 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 456 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tighten

Simulation Results Reference

Results Folder: /Users/cbarrows/Documents/repos/SIIPExamples.jl/RTS-GMLC-master/rts-test/7/results

Reference Tables

________________

stage-UC

P__ThermalStandard: dataframe size (1, 3)

P__RenewableDispatch: dataframe size (1, 3)

Reg_Up__VariableReserve_PowerSystems.ReserveUp: dataframe size (1, 3)

Flex_Up__VariableReserve_PowerSystems.ReserveUp: dataframe size (1, 3)

parameter_service_requirement_VariableReserve_PowerSystems.ReserveDown: dataframe size (1, 3)

parameter_service_requirement_VariableReserve_PowerSystems.ReserveUp: dataframe size (1, 3)

parameter_P_PowerLoad: dataframe size (1, 3)

P__HydroEnergyReservoir: dataframe size (1, 3)

Flex_Down__VariableReserve_PowerSystems.ReserveDown: dataframe size (1, 3)

Spin_Up_R1__VariableReserve_PowerSystems.ReserveUp: dataframe size (1, 3)

parameter_P_RenewableDispatch: dataframe size (1, 3)

parameter_P_HydroEnergyReservoir: dataframe size (1, 3)

Spin_Up_R3__VariableReserve_PowerSystems.ReserveUp: dataframe size (1, 3)

parameter_P_RenewableFix: dataframe size (1, 3)

On__ThermalStandard: dataframe size (1, 3)

parameter_P_HydroDispatch: dataframe size (1, 3)

Reg_Down__VariableReserve_PowerSystems.ReserveDown: dataframe size (1, 3)

Start__ThermalStandard: dataframe size (1, 3)

Stop__ThermalStandard: dataframe size (1, 3)

Spin_Up_R2__VariableReserve_PowerSystems.ReserveUp: dataframe size (1, 3)

stage-ED

parameter_P_HydroDispatch: dataframe size (24, 3)

parameter_P_RenewableDispatch: dataframe size (24, 3)

P__ThermalStandard: dataframe size (24, 3)

P__RenewableDispatch: dataframe size (24, 3)

parameter_P_HydroEnergyReservoir: dataframe size (24, 3)

parameter_P_RenewableFix: dataframe size (24, 3)

parameter_P_PowerLoad: dataframe size (24, 3)

P__HydroEnergyReservoir: dataframe size (24, 3)

γ⁺__P: dataframe size (24, 3)

γ⁻__P: dataframe size (24, 3)

stage-UC

time length: 24

stage-ED

time length: 12

## Results

In [13]:
uc_results = load_simulation_results(sim_results, "UC");
ed_results = load_simulation_results(sim_results, "ED");

## Plotting
Take a look at the examples in [the plotting folder.](../../notebook/PowerSimulations_examples/Plotting)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*